In [184]:
!pip install natasha
!pip install spacy
!pip install flair

In [185]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)




from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from pymystem3 import Mystem
my = Mystem()
import spacy
import flair
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('pos')
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.metrics import accuracy_score
import re

2020-10-18 19:52:54,473 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [186]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-18 19:52:54--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.245, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-18 19:52:55--  http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)... 5.45.220.12, 2a02:6b8:0:2002::13
Connecting to cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)|5.45.220.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.2’

mystem-3.0-linux3.1 100%[===================>] 

In [187]:
def clean(text):
  for i in punctuation:
    text = text.replace(i, '')
  return text.lower()

In [188]:
with open('ru_text.txt', 'r', encoding='utf-8') as f:
  for_syntax = f.read()
with open('ru_text.txt', 'r', encoding='utf-8') as f:
  ru_text = clean(f.read()).split('\n')
with open('eng_text.txt', 'r', encoding='utf-8') as f:
  eng_text = clean(f.read()).split('\n')

In [241]:
ru_text

['сложно самому написать текст для парсера ',
 'это словно забираться на тяньшань с закрытыми глазами ',
 'справится джеки чан чак норрис или михаил пореченков ',
 'водить машину с мкпп сложнее чем с акпп',
 'мы не ели много мороженного но смотрели на ели ',
 'мой кит очень усталый говорит что ему плохо',
 'петя сделал качественный коммит на гитхабе читая книгу толстого на покетбуке',
 'согласно данным сми на митинге побывало около четырех тыс человек',
 'крокодил крокожу и буду крокодить',
 'навстречу мне двигалось сто вась крича и прыгая',
 'вдруг явился посол и начал танцевать яростно',
 'на паре сидели пятьдесят студентов и они обсуждали множество непонятных и неоднозначных вопросов',
 'это будет последнее предложение это не странно',
 'петя взял стакан из стекла а по нему стекла вода',
 'я положил пирожки в печь и начал их печь',
 'задание не простое а очень сложное']

Сложности текста: омонимы ели мороженное - смотрели на ели, стакан из стекла - стекла вода, аббривеатуры МКПП АКПП, выдуманные слова крокожу, буду крокодить, книга Толстого - Толстого может быть сущ или прил

In [242]:
eng_text

['i am very hungry i want to eat pizza',
 'this is a text for the parsing',
 'mr smith took the drink and they drink it',
 'she loves to swim and she do it very well',
 'mrs smith loves to smith because she is a smith',
 'however bob has eaten one green orange',
 'he asked me not very calmly about my new laptop',
 'the final match of the uefa football league is today',
 'mary thinked about orange juice',
 'all the people are in danger',
 'mr brown throwed the ring in the river and she had to ring the bell']

Сложности: orange как апельсин и как оранжевый, аббривиатура UEFA, to ring и ring кольцо, три слова smith - два глагола и одно существительное

In [192]:
ru_words = []
for i in ru_text:
  for j in clean(i).split():
    ru_words.append(j)

In [193]:
eng_words = []
for i in eng_text:
  for j in clean(i).split():
    eng_words.append(j)

In [194]:
ru_tags = ['ADV PRON VERB NOUN PREP NOUN',
           'PRON CONJ VERB PREP NOUN PREP ADJ NOUN',
           'VERB NOUN NOUN NOUN NOUN PREP NOUN NOUN',
           'VERB NOUN PREP NOUN ADJ CONJ PREP NOUN',
           'PRON PART VERB ADV NOUN CONJ VERB PREP NOUN',
           'PRON NOUN ADV ADJ VERB CONJ PRON ADV',
           'NOUN VERB ADJ NOUN PREP NOUN VERB NOUN NOUN PREP NOUN',
           'PREP NOUN NOUN PREP NOUN VERB ADV NUM NUM NOUN',
           'NOUN VERB CONJ VERB VERB',
           'ADV PRON VERB NUM NOUN VERB CONJ VERB',
           'ADV VERB NOUN CONJ VERB VERB ADV',
           'PREP NOUN VERB NUM NOUN CONJ PRON VERB NOUN ADJ CONJ ADJ NOUN',
           'PRON VERB ADJ NOUN PRON PART ADV',
           'NOUN VERB NOUN PREP NOUN CONJ PREP PRON VERB NOUN',
           'PRON VERB NOUN PREP NOUN CONJ VERB PRON VERB',
           'NOUN PART ADJ CONJ ADV ADJ'
           ]

In [195]:
eng_tags = ['PRON VERB ADV ADJ PRON VERB PART VERB NOUN',
            'DET VERB DET NOUN PREP DET NOUN',
            'NOUN NOUN VERB DET NOUN CONJ PRON VERB PRON',
            'PRON VERB PART VERB CONJ PRON VERB PRON ADV ADJ',
            'NOUN NOUN VERB PART VERB CONJ NOUN VERB DET NOUN',
            'CONJ NOUN VERB VERB NUM ADJ NOUN',
            'PRON VERB PRON PART ADV ADJ PREP PRON ADJ NOUN',
            'DET ADJ NOUN PREP DET NOUN ADJ NOUN VERB NOUN',
            'PRON VERB PREP ADJ NOUN',
            'PRON DET NOUN VERB PREP NOUN',
            'NOUN NOUN VERB DET NOUN PREP DET NOUN CONJ PRON VERB VERB VERB DET NOUN'
            ]

In [196]:
ru_tags_all = []
for i in ru_tags:
  for j in i.split():
    ru_tags_all.append(j)
eng_tags_all = []
for i in eng_tags:
  for j in i.split():
    eng_tags_all.append(j)

In [198]:
def pym_normalized(text):
  text = re.sub(r'ADJ\w', 'ADJ', text)
  text = text.replace('COMP', 'ADJ')
  text = text.replace('NUMR', 'NUM')
  text = text.replace('ADVB', 'ADV')
  text = text.replace('INFN', 'VERB')
  text = text.replace('GRND', 'VERB')
  text = text.replace('NPRO', 'PRON')
  text = text.replace('PRCL', 'PART')
  if text is None:
    text = ''
  return text

In [199]:
pym_tags = []
for i in ru_words:
  pym_tags.append(pym_normalized(morph.parse(i)[0].tag.POS))

In [201]:
def mys_normalized(text):
  text = re.sub('\w*PRO\w*', 'PRON', text)
  text = re.sub('^A$', 'ADJ', text)
  text = re.sub('^V$', 'VERB', text)
  text = re.sub('^S$', 'NOUN', text)
  text = re.sub('^PR$', 'PREP', text)
  return text

In [202]:
mys_tags = []
for i in ru_words:
  if my.analyze(i)[0]['analysis']:
    mys_tags.append(mys_normalized(re.search(
        r'[A-Z]+',my.analyze(i)[0]['analysis'][0]['gr']).group()))
  else:
    print(i)
    mys_tags.append('')

мкпп


In [204]:
def nts_normalized(tag):
  tag = re.sub('PROPN', 'NOUN', tag)
  tag = re.sub('ADP', 'PREP', tag)
  tag = re.sub('\wCONJ', 'CONJ', tag)
  tag = re.sub('AUX', 'VERB', tag)
  return tag

In [205]:
nts_tags = []
text = ''
for i in ru_text:
  text += clean(i) + ' '
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for i in doc.tokens:
  nts_tags.append((nts_normalized(i.pos)))

# Точность русских теггеров

In [207]:
print('Pymorphy: ', accuracy_score(pym_tags, ru_tags_all))
print('Mystem: ', accuracy_score(mys_tags, ru_tags_all))
print('Natasha: ', accuracy_score(nts_tags, ru_tags_all))

Pymorphy:  0.8872180451127819
Mystem:  0.849624060150376
Natasha:  0.9022556390977443


In [208]:
def spc_normalized(tag):
  tag = tag.replace('ADP', 'PREP')
  tag = tag.replace('PROPN', 'NOUN')
  tag = tag.replace('AUX', 'VERB')
  tag = re.sub(r'\wCONJ', 'CONJ', tag)
  return tag

In [209]:
spc_tags = []
nlp = spacy.load("en_core_web_sm")
eng_str = ''
for i in eng_text:
  eng_str += i + ' '
doc = nlp(eng_str)
for token in doc:
  spc_tags.append(spc_normalized(token.pos_))

In [211]:
flair_tags = []
sent = Sentence(eng_str)
tagger.predict(sent)
tags_str = sent.to_tagged_string()

In [213]:
def flr_normalized(tag):
  tag = re.sub(r'PRP.?', 'PRON', tag)
  tag = re.sub(r'VB\w?', 'VERB', tag)
  tag = re.sub(r'^RB$', 'ADV', tag)
  tag = tag.replace('TO', 'PART')
  tag = re.sub(r'NN\w?', 'NOUN', tag)
  tag = tag.replace('DT', 'DET')
  tag = tag.replace('JJ', 'ADJ')
  tag = tag.replace('CC', 'CONJ')
  tag = tag.replace('WRB', 'CONJ')
  tag = tag.replace('IN', 'PREP')
  tag = tag.replace('UH', 'NOUN')
  tag = tag.replace('PDT', 'PRON')
  tag = tag.replace('CD', 'NUM')
  return tag

In [214]:
flr_tags = [flr_normalized(x) for x in re.findall(r'<(\w*.?)>', tags_str)]

In [216]:
nltk_tags = []
text = word_tokenize(eng_str)
tags = nltk.pos_tag(text)
for i in tags:
  nltk_tags.append(flr_normalized(i[1]))

# Точность английских теггеров

In [244]:
print('Spacy: ', accuracy_score(spc_tags, eng_tags_all))
print('Flair: ', accuracy_score(flr_tags, eng_tags_all))
print('NLTK: ', accuracy_score(nltk_tags, eng_tags_all))

Spacy:  0.8673469387755102
Flair:  0.8775510204081632
NLTK:  0.8163265306122449


In [218]:
ru_str = ''
for i in ru_text:
  ru_str += i + ' '
ru_str = ru_str.replace('  ', ' ')

# Для второй части

In [235]:
def make_word_list(text):
  word_list = clean(text).replace('\n', ' ').split()
  return word_list

In [256]:
def make_pos(word_list):
  tags_list = []
  for w in word_list:
    doc = Doc(w)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    tags_list.append((nts_normalized(doc.tokens[0].pos)))
  return tags_list

In [237]:
def find_bigrams(words_list, tags_list):
  not_verb = []
  adj_noun = []
  adv_adj = []
  for i in range(len(words_list)):
    try:
      if words_list[i] == 'не' and tags_list[i+1] == 'VERB':
        not_verb.append(words_list[i] + ' ' + words_list[i+1])
      if tags_list[i] == 'ADJ' and tags_list[i+1] == 'NOUN':
        adj_noun.append(words_list[i] + ' ' + words_list[i+1])
      if tags_list[i] == 'ADV' and tags_list[i+1] == 'ADJ':
        adv_adj.append(words_list[i] + ' ' + words_list[i+1])
    except IndexError:
      pass
  return not_verb, adj_noun, adv_adj